# Benchmark of 4D phase space reconstruction algorithms 

In [ ]:
import sys
import importlib
import numpy as np
import pandas as pd
from skimage import filters
from matplotlib import pyplot as plt
import proplot as pplt
from tqdm.notebook import tqdm
from tqdm.notebook import trange

import reconstruct as rec
from tools import ap
from tools import analysis as ba
from tools import plotting as mplt
from tools import utils

In [ ]:
pplt.rc['grid'] = False
pplt.rc['cmap.sequential'] = 'dusk_r'
pplt.rc['cmap.discrete'] = False
pplt.rc['cmap.diverging'] = 'vlag'
pplt.rc['figure.facecolor'] = 'white'
pplt.rc['grid.alpha'] = 0.04

## Setup 

Load the bunch. (See https://www.dropbox.com/sh/1f2yoh5n1wrgfxm/AAB0C__P9cKjcz7YzOA7iqmsa?dl=0). Columns are orded as {x (mm), x' (mrad), y (mm), y' (mrad), z (mm), dE (MeV)}.

In [ ]:
# filename = 'data/coords_SNS_elliptical_painting_300turns.npy'
filename = 'data/2022-07-01_run_MEBT123_HZ04_VT34a_bunch.npy'

X = np.load(filename) 
X = X[:, :4]  # drop longitudinal coordinates
print(X.shape)

Take random sample of bunch particles.

In [ ]:
X = utils.rand_rows(X, int(0.1 * X.shape[0]))

Normalize $x$-$x'$ and $y$-$y'$ using the statistical 2D Twiss parameters.

In [ ]:
# Compute statistical Twiss parameters.
Sigma = np.cov(X.T)
alpha_x, alpha_y, beta_x, beta_y = ba.twiss2D(Sigma)
print('Statistical Twiss parameters:')
print('  alpha_x = {}'.format(alpha_x))
print('  alpha_y = {}'.format(alpha_y))
print('  beta_x = {}'.format(beta_x))
print('  beta_y = {}'.format(beta_y))

# Define the lattice Twiss parameters. (If `dalpha` and `dbeta` are nonzero, 
# the lattice Twiss parameters will be different than the statistical Twiss
# parameters. 
state = np.random.RandomState()
state.seed(17)
dalpha = 0.0
dbeta = 0.0 * max(beta_x, beta_y)
alpha_x += np.random.uniform(-dalpha, dalpha)
alpha_y += np.random.uniform(-dalpha, dalpha)
beta_x += np.random.uniform(-dbeta, dbeta)
beta_y += np.random.uniform(-dbeta, dbeta)
print('Lattice Twiss parameters:')
print('  alpha_x = {}'.format(alpha_x))
print('  alpha_y = {}'.format(alpha_y))
print('  beta_x = {}'.format(beta_x))
print('  beta_y = {}'.format(beta_y))

# Normalize the distribution using the statistical Twiss parameters.
V = ap.norm_matrix_4x4_uncoupled(alpha_x, alpha_y, beta_x, beta_y)
Vinv = np.linalg.inv(V)
Xn = utils.apply(Vinv, X)

Bin the normalized coordinates at the reconstruction location.

In [ ]:
n_bins = 50
limits_rec = mplt.auto_limits(Xn, sigma=3.25)
dims = ["x", "x'", "y", "y'"]
units = ["mm", "mrad", "mm", "mrad"]
labels = [f"{dim} [{unit}]" for dim, unit in zip(dims, units)]

axes = mplt.corner(Xn, kind='hist', diag_kind='line', labels=labels, limits=limits_rec, bins=n_bins)
axes.format(xlocator=('maxn', 3), ylocator=('maxn', 3))
plt.show()

In [ ]:
f_true, edges_rec = np.histogramdd(Xn, n_bins, limits_rec, density=True)
grid_rec = [rec.get_bin_centers(_edges) for _edges in edges_rec]
bin_volume = rec.get_bin_volume(limits_rec, n_bins)

View projections interactively.

In [ ]:
mplt.interactive_proj1d(f_true, kind='step', dims=dims, units=units, coords=grid_rec)

In [ ]:
mplt.interactive_proj2d(f_true, dims=dims, units=units, coords=grid_rec)

## Hock's method

Transport the distribution to the screen. Since the horizontal and vertical optics are varied independently, we can transport $x$-$x'$ and $y$-$y'$ independently. Assume the phase advances are evenly spaced over 180 degrees. The Twiss parameters at the screen are randomly varied about their nominal values on each iteration.

In [ ]:
def make_tmats(phase_advances, betas, alphas, Vinv):
    """Return list of 2x2 transfer matrices."""
    tmats = []
    for phase_adv, beta, alpha in zip(phase_advances, betas, alphas):
        P = utils.rotation_matrix(phase_adv)
        V_screen = ap.norm_matrix(alpha, beta)
        tmats.append(np.linalg.multi_dot([V_screen, P, Vinv]))
    return tmats

In [ ]:
K = 12  # number of horizontal optics settings
L = 12  # number of vertical optics settings

phase_adv_x = np.linspace(0.0, np.pi, K, endpoint=False)
betas_x = np.random.uniform(1.0, 1.0, size=K)
alphas_x = np.random.uniform(0.0, 0.0, size=K)

phase_adv_y = np.linspace(0.0, np.pi, L, endpoint=False)
betas_y = np.random.uniform(1.0, 1.0, size=L)
alphas_y = np.random.uniform(0.0, 0.0, size=L)

# Store list of x and y coordinates only for each run.
tmats_x = make_tmats(phase_adv_x, betas_x, alphas_x, Vinv[:2, :2])
tmats_y = make_tmats(phase_adv_y, betas_y, alphas_y, Vinv[2:, 2:])
print('Transporting x-x.')
xx_list = [utils.apply(Mx, X[:, :2])[:, 0] for Mx in tqdm(tmats_x)]
print('Transporting y-y.')
yy_list = [utils.apply(My, X[:, 2:])[:, 0] for My in tqdm(tmats_y)]

Bin the coordinates on the screen. Below, I assume we stay in normalized phase space so that the transfer matrices are just rotations. In a more realistic simulation, we will need to specify the screen dimensions.

In [ ]:
xmax_screen = edges_rec[0]
ymax_screen = edges_rec[1]
edges_meas = [xmax_screen, ymax_screen]
grid_meas = [rec.get_bin_centers(_edges) for _edges in edges_meas]

S = np.zeros((n_bins, n_bins, K, L))
for k, xx in enumerate(tqdm(xx_list)):
    for l, yy in enumerate(yy_list):
        S[:, :, k, l], _, _ = np.histogram2d(xx, yy, edges_meas)

In [ ]:
fig, axes = pplt.subplots(nrows=K, ncols=L, figwidth=10, space=0.175)
for i in range(axes.shape[0]):
    for j in range(axes.shape[1]):
        ax = axes[i, j]
        ax.pcolormesh(S[:, :, i, j].T, cmap='mono_r', ec='None')
axes.format(
    xticks=[], yticks=[], 
    xlabel='x', ylabel='y',
    suptitle='simulated images',
)
plt.show()

We'll reconstruct in normalized phase space.

In [ ]:
tmats_x_n = [np.matmul(Mx, V[:2, :2]) for Mx in tmats_x]
tmats_y_n = [np.matmul(My, V[2:, 2:]) for My in tmats_y]

In [ ]:
f_hock = rec.hock4D(S, grid_meas, (grid_rec[0], grid_rec[2]),
                    tmats_x_n, tmats_y_n, method='SART', iterations=3)
f_hock = rec.process(f_hock, keep_positive=True, density=True, limits=limits_rec)

In [ ]:
print('min(f) = {}'.format(np.min(f_hock)))
print('max(f) = {}'.format(np.max(f_hock)))
print('sum(f) * bin_volume = {}'.format(np.sum(f_hock) * bin_volume))
print()
print('min(f_true) = {}'.format(np.min(f_true)))
print('max(f_true) = {}'.format(np.max(f_true)))
print('sum(f_true) * bin_volume = {}'.format(np.sum(f_true) * bin_volume))

In [ ]:
def plot_compare_1D(f1, f2):
    fig, axes = pplt.subplots(ncols=6, nrows=3, figwidth=8, space=0.5)
    inds = [(0, 2), (0, 1), (2, 3), (0, 3), (2, 1), (1, 3)]
    for j, ind in enumerate(inds):
        x = grid_rec[ind[0]]
        y = grid_rec[ind[1]]
        H1 = rec.project(f1, ind)
        H2 = rec.project(f2, ind)
        Hdiff = H1 - H2
        # Need to plot absolue difference `np.abs(Hdiff)` if doing log plot.
        for ax, H in zip(axes[:, j], [H1, H2, Hdiff]):
            mplt.plot_image(H, x=x, y=y, ax=ax, 
                            # norm='log', handle_log='floor', 
                            ec='None')
        axes[0, j].format(title=f'{dims[ind[0]]}-{dims[ind[1]]}')
    axes.format(xticks=[], yticks=[], leftlabels=['Reconstructed', 'True', 'Difference'])
    plt.show()

In [ ]:
axes = plot_compare_1D(f_hock, f_true)

In [ ]:
f_err = f_true - f_hock
mplt.interactive_proj1d(f_err, kind='line', dims=dims, units=units, coords=grid_rec)

In [ ]:
mplt.interactive_proj2d(f_hock, dims=dims, units=units, coords=grid_rec)

## 4D ART 

The maximum reconstruction grid resolution for this method is approximately 50, and that will take hours.

In [ ]:
n_bins = 25
f_true2, edges_rec = np.histogramdd(Xn, n_bins, limits_rec, density=True)
grid_rec = [rec.get_bin_centers(_edges) for _edges in edges_rec]
bin_volume = rec.get_bin_volume(limits_rec, n_bins)

In [ ]:
edges_meas = [edges_rec[0], edges_rec[2]]
grid_meas = [rec.get_bin_centers(_edges) for _edges in edges_meas]

Simulate measurements with the new screen resolution and number of projections.

In [ ]:
K = 8  # number of horizontal optics settings
L = 8  # number of vertical optics settings

phase_adv_x = np.linspace(0.0, np.pi, K, endpoint=False)
betas_x = np.random.uniform(1.0, 1.0, size=K)
alphas_x = np.random.uniform(0.0, 0.0, size=K)

phase_adv_y = np.linspace(0.0, np.pi, L, endpoint=False)
betas_y = np.random.uniform(1.0, 1.0, size=L)
alphas_y = np.random.uniform(0.0, 0.0, size=L)

tmats_x = make_tmats(phase_adv_x, betas_x, alphas_x, Vinv[:2, :2])
tmats_y = make_tmats(phase_adv_y, betas_y, alphas_y, Vinv[2:, 2:])
print('Transporting x-x')
xx_list = [utils.apply(Mx, X[:, :2])[:, 0] for Mx in tqdm(tmats_x)]
print('Transporting y-y')
yy_list = [utils.apply(My, X[:, 2:])[:, 0] for My in tqdm(tmats_y)]

In [ ]:
tmats_x_n = [np.matmul(Mx, V[:2, :2]) for Mx in tmats_x]
tmats_y_n = [np.matmul(My, V[2:, 2:]) for My in tmats_y]
projections, tmats_n = [], []
for xx, Mx in tqdm(zip(xx_list, tmats_x_n)):
    for yy, My in zip(yy_list, tmats_y_n):
        projection, _, _ = np.histogram2d(xx, yy, edges_meas)
        projections.append(projection)
        M = np.zeros((4, 4))
        M[:2, :2] = Mx
        M[2:, 2:] = My
        tmats_n.append(M)

Launch the reconstruction.

In [ ]:
f_art = rec.art4D(projections, tmats_n, grid_rec, grid_meas)
f_art = rec.process(f_art, keep_positive=True, density=True, limits=limits_rec)

In [ ]:
pplt.__version__

In [ ]:
axes = plot_compare_1D(f_art, f_true2)

In [ ]:
mplt.interactive_proj2d(f_art, dims=dims, coords=grid_rec, units=units)